In [17]:
import logic as lc
#Setting up the environement
import math as m
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Dummy pandas dataframe
#Setting up time in minutes
start = 0
end = 480
def minute_range(start, end, step=15):
    cur = start
    time_list = [cur]
    while cur < end:
        cur = cur+step
        time_list.append(cur)
    return time_list
        

#numoy array
max = 50
time_list = minute_range(start,end,15)
x = np.linspace(0,m.pi*50,len(time_list))
y = np.array([m.sin(i/50)*max for i in x])
time = time_list

df = pd.DataFrame({'time':time_list})

def eight_row_sin(lst, df, max_, min_, sd,x):
    max_ = np.random.uniform(min_,max_, 1)
    def change_to_zero(x):
        if x<0:
            return 0
        else:
            return x
    
    
    for i in lst:
        df[i]  = np.array([m.sin(i/50)*max_ for i in x])
        df[i] = df[i].apply(lambda x: int(np.random.uniform(x - sd, x + sd, 1)))
        df[i] = df[i].apply(change_to_zero)
    return df


Day_Leg = ['North_a','South_a','West_a','East_a']
df = eight_row_sin(lst=Day_Leg, df=df, max_=200, min_=150, sd = 30, x=x)



# for i in Day_Leg:
#     fig,ax = plt.subplots()
#     print(i)
#     ax.scatter(df.x,df[i])
#     ax.set(xlabel = 'Time (Minutes)', ylabel = 'Demand')
#     plt.show()


In [32]:
time_index_all_routes = lc.load_time_index(df)
possible_route = {
    "North_a":{
        "South_f": 40, 
        "West_f": 60},
    "South_a":{
        "North_f": 75, 
        "East_f": 25},
    "East_a":{
        "West_f": 70,
        "North_f": 30},
    "West_a":{
        "East_f": 55,
        "South_f": 45}
    }

reference_dict = {
    "North_a" : "785902529",
    "North_f" : "-785902529",
    "West_a" : "785902540#0",
    "West_f" : "-785902540#1",
    "South_a" : "-732022517",
    "South_f" : "732022517",
    "East_a" : "-785902540#2",
    "East_f" : "785902540#2"}

In [34]:
PATH = "Main Route Simulation\osm.passenger.trips.xml"
def generate(route_weight,reference_dict, df, time_index):
    """ 
    Generates the trips in xml of the route
    """ 
    #generates the tag of each vehicle in xml format
    vehicle_xml_tag_array = []
    count = 0
    for lane_approach in route_weight:
        lane_approach_dict = route_weight[lane_approach]

        for index,value in enumerate(time_index[lane_approach]):
            tag = '<trip id="veh{}" type="veh_passenger" depart="{}" departLane="best" from="{}" to="{}"/>\n'.format(count,value,reference_dict[lane_approach],reference_dict[lc.to_route(lane_approach, possible_route=route_weight)])
            vehicle_xml_tag_array.append((tag, value))
            count +=1
    vehicle_xml_tag_array = sorted(vehicle_xml_tag_array,key = lambda x: x[1])  #order keys using the seconds tuple which is the duration(keys, duration)
    vehicle_xml_tag_array = [value for value, _ in vehicle_xml_tag_array]

    #writes the schema and vehicle type of the network as ...trips.xml
    with open(PATH, "w") as file:
        print(r'''<?xml version="1.0" encoding="UTF-8"?>

<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">
<vType id="veh_passenger" vClass="passenger"/>



''', file=file)
    
    #appends the tags to ...trips.xml 
    with open(PATH, "a") as file:
        for tag in vehicle_xml_tag_array:
            file.write(tag)
        file.write("</routes>")

all_vehicles = generate(possible_route,reference_dict,df,time_index_all_routes)